In [1]:
import pandas as pd

import numpy as np
from datetime import datetime

#### LIMPIEZA

In [5]:
class Limpieza:
    #Inicializamos los atributos
    def __init__(self, enlace):
        self.df = pd.read_excel(enlace, sheet_name='Hoja1', header=0)

        
    def to_category(self, df):
        l_columns = ['CliCod','OpeTipo','Suplemento','ComCobCia','CodOpe','Operacion','Seccion','Moneda','ProdCod','ProdNombre','ProcoCod','Compania']
        for column in l_columns:
            df[column] = df[column].astype('category')
        return df
  
    
    def nuevas_columnas(self, df):
        df['Operacion'] = df['Operacion'].str.strip()
        df['Compania'] = df['Compania'].str.strip()
        df['Año']= df['VigDesde'].map(lambda x: x.year)
        df['Seccion'].replace('ART','A.R.T.', inplace=True)
        df.sort_values(by=['Año'],inplace= True)
        return df

    
    def filtrado_un_mes(self, df, year, fecha_inicio, fecha_fin):
        return df[df['FecPedido'].between(datetime.strptime(fecha_inicio.format(year),"%Y-%m-%d"),
                                          datetime.strptime(fecha_fin.format(year),"%Y-%m-%d"))
                  ]
    
    
    def filtrado_un_anio(self, df, year):
        # Filter by month
        diciembre = self.filtrado_un_mes(df, year, '{}-12-01','{}-12-31')
        noviembre = self.filtrado_un_mes(df, year, '{}-11-01','{}-11-30')
        octubre = self.filtrado_un_mes(df, year, '{}-10-01','{}-10-31')
        septiembre = self.filtrado_un_mes(df, year, '{}-09-01','{}-09-30')
        agosto = self.filtrado_un_mes(df, year, '{}-08-01','{}-08-31')
        julio = self.filtrado_un_mes(df, year, '{}-07-01','{}-07-31')
        junio = self.filtrado_un_mes(df, year, '{}-06-01','{}-06-30')
        mayo = self.filtrado_un_mes(df, year, '{}-05-01','{}-05-31')
        abril = self.filtrado_un_mes(df, year, '{}-04-01','{}-04-30')
        marzo = self.filtrado_un_mes(df, year, '{}-03-01','{}-03-31')
        febrero = self.filtrado_un_mes(df, year, '{}-02-01','{}-02-28')
        enero = self.filtrado_un_mes(df, year, '{}-01-01','{}-01-31')

        # Dictionary with month date and DataFrame filtered
        d_year = {'{}-01'.format(year):enero, '{}-02'.format(year):febrero, 
                  '{}-03'.format(year):marzo, '{}-04'.format(year):abril, 
                  '{}-05'.format(year):mayo, '{}-06'.format(year):junio,
                  '{}-07'.format(year):julio, '{}-08'.format(year):agosto, 
                  '{}-09'.format(year):septiembre, '{}-10'.format(year):octubre, 
                  '{}-11'.format(year):noviembre, '{}-12'.format(year):diciembre}
        return d_year
    
    
    def filtrado_operacion(self, df, operacion, ramo):
        if ramo == None:
            return df[(df['Operacion'].isin(operacion))]
        else:
            return df[(df['Operacion'].isin(operacion)) & (df['Seccion'] == ramo)]
    
    
    def operaciones(self, df, ramo = None):
        r = ramo
        contenedor = {'emisiones': None, 'renovaciones':None , 'reno_refa':None , 'anulaciones':None,'otros':None }
        operaciones = (['Emisión Nueva'], 
                       ['Renovación Póliza'], 
                       ['Renovación Póliza', 'Endoso de Refacturación'], 
                       ['N/C de Anulación Póliza'],
                       ['Endoso de Corrección de Datos', 'Endoso Motivos Varios', 'N/C de Rebaja', 'N/C de Anulación Endoso', 
                        'Endoso de Inclusión de Item', 'N/C de Baja de Item','Endoso de Rehabilitación'] 
                       )
        for c, o in zip(contenedor, operaciones ):
            contenedor[c] = self.filtrado_operacion(df, o, ramo = ramo)
        return contenedor
        
    
    
    def operaciones_Anio(self):
        """
        Create a file with sum of the operation by month in the broker.
            - Emisiones
            - Renovaciones
            - Refa_Reno
            - Anulaciones
            - Otros_Endosos
        """
        base = {'Fecha':[], 'Emisiones': [], 'Renovaciones':[], 'Refa_Reno':[], 
                'Anulaciones':[], 'Otros_Endosos':[],'Año':[],}
        
        df = self.to_category(self.df)                # A: Columns converted to Category to optimize ram - 7,8mb to 5,7mb -
        df = self.nuevas_columnas(df)                 # A: Column 'Año' and 'Seccion' created, others cleaned
        l_year = df['Año'].unique()                   # U: List to iterate thru years
        
        for year in l_year:
            d_year = self.filtrado_un_anio(df, year)  # U: Get a dictionary with the year filtered by month 

            for mes in d_year:
                contenedor = self.operaciones(d_year[mes])         # A: Get a dict with one dataframe by operation
                
                tmp = (mes,len(contenedor['emisiones']),len(contenedor['renovaciones']),    # A: List with the Nº of each operations
                           len(contenedor['reno_refa']),len(contenedor['anulaciones']),
                           len(contenedor['otros']),year)                                 
                
                for column, value in zip(base,tmp):    
                    base[column].append(value)          # A: Save the numbers to create the final Data Frame
                    
        df_muestra = pd.DataFrame(base)
        df_muestra.to_csv('../0_Files/Procesados/Consolidado_Operaciones.csv',index=False)
        
        
    def operaciones_Anio_Ramo(self):
        """
        Create a file with sum of the operation by month and insurance type in the broker.
            - Emisiones
            - Renovaciones
            - Refa_Reno
            - Anulaciones
            - Otros_Endosos
        """
        base = {'Fecha':[], 'Emisiones': [], 'Renovaciones':[], 'Refa_Reno':[],
                'Anulaciones':[], 'Otros_Endosos':[], 'Ramo':[], 'Año':[], }
        
        df = self.to_category(self.df)                # A: Columns converted to Category to optimize ram - 7,8mb to 5,7mb -
        df = self.nuevas_columnas(self.df)            # A: Column 'Año' and 'Seccion' created, others cleaned
        l_year = df['Año'].unique()              # U: List to iterate thru years
        l_ramos = df['Seccion'].unique()         # U: List to iterathe thru insurance type
        
        for year in l_year:
            d_year = self.filtrado_un_anio(df, year)  # U: Get a dictionary with the year filtered by month 
            
            for ramo in l_ramos:
                
                for mes in d_year:
                    contenedor = self.operaciones(d_year[mes], ramo = ramo)
                    
                    tmp = (mes,len(contenedor['emisiones']),len(contenedor['renovaciones']),    # A: List with the Nº of each operations
                               len(contenedor['reno_refa']),len(contenedor['anulaciones']),
                               len(contenedor['otros']), ramo, year) 
                    
                    for column, value in zip(base,tmp):    
                        base[column].append(value)          # A: Save the numbers to create the final Data Frame
                        
        df_muestra = pd.DataFrame(base)
        df_muestra.to_csv('../0_Files/Procesados/Consolidado_Operaciones_Ramo.csv',index=False)
    

In [6]:
l = Limpieza('../0_Files/Brutos/2016-2018_Detalle_de_Produccion_TODOS.xls')
l.operaciones_Anio()
l.operaciones_Anio_Ramo()